<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/PartisanBiasDetection/LocalScraperTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This is the automated scraper for FoxNews(Digital)**

When addapting this script for other news sources you will need to:
*   Change the uril point you to the rss feed or breaking news webpage
*   Change the suffix in the parameter section

In [123]:
#@title Install Dependancies
!pip install requests beautifulsoup4 --quiet

#Import libraries
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime,timezone
import pytz
from pathlib import Path

est_tz = pytz.timezone('US/Eastern')

In [70]:
# @title Define Utility Functions

#Article Scrapper
def scrape_article_text(url):
    # Send a GET request to the URL
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find elements containing text - this part may need customization
        # For example, assuming article text is within <p> tags
        article_text = ' '.join(p.get_text() for p in soup.find_all('p'))
        return article_text
    else:
        return "Error: Unable to fetch the webpage."

#Article Scraper with User Agent - add to utility functions
def scrape_article_text_useragent(url,ua):
    # Send a GET request to the URL
    response = requests.get(url, headers=ua)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find elements containing text - this part may need customization
        # For example, assuming article text is within <p> tags
        article_text = ' '.join(p.get_text() for p in soup.find_all('p'))
        return article_text
    else:
        return "Error: Unable to fetch the webpage."


In [63]:
#Parameter specification
sourceSuffix = 'FoxNews'

In [67]:
#Fox News Article Text Link Scraper
url = 'https://moxie.foxnews.com/google-publisher/politics.xml'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, "lxml")
links = []
artTxt = []
for item in soup.find_all("item"):
  link= str(item)
  i = link.find("<link/>")
  j = link.find("<guid")
  x = link[i+7:j]
  y = x.split('\n', 1)[0]
  links.append(y)
  article_text = scrape_article_text(y)
  artTxt.append(article_text)

#Fox News Article Header Scraper
headers = []
for item in soup.find_all("item"):
  header= str(item)
  i = header.find("<title>")
  j = header.find("</title>")
  x = header[i+7:j]
  y = x.split('\n', 1)[0]
  headers.append(y)

#Fox News Article PubDate Scraper
pubds = []
for item in soup.find_all("item"):
  pubd= str(item)
  i = pubd.find("<pubdate>")
  j = pubd.find("</pubdate>")
  x = pubd[i+9:j]
  y = x.split('\n', 1)[0]
  pubds.append(y)

d = {'Links': links, 'ArticleHeaders': headers, 'ArticleText': artTxt, 'PubDate': pubds}
artD = pd.DataFrame(d)


<ipython-input-67-3c27d6710d64>:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html_page.text, "lxml")


In [162]:
# Define event times (UTC), then convert to EST
event_time_utc_0 = datetime(9999, 12, 31, 5, 0, 0, tzinfo=pytz.utc)  # Event before in UTC
event_time_utc_6 = datetime(9999, 12, 31, 11, 0, 0, tzinfo=pytz.utc)     # Event after in UTC
event_time_utc_12 = datetime(9999, 12, 31, 16, 0, 0, tzinfo=pytz.utc)  # Event before in UTC
event_time_utc_16 = datetime(9999, 12, 31, 21, 0, 0, tzinfo=pytz.utc)     # Event after in UTC
event_time_utc_EoD = datetime(9999, 12, 31, 4, 59, 0, tzinfo=pytz.utc)     # Event after in UTC

# Convert to EST and extract time
event_time_est_0 = event_time_utc_0.astimezone(est_tz).time()
event_time_est_6 = event_time_utc_6.astimezone(est_tz).time()
event_time_est_12 = event_time_utc_12.astimezone(est_tz).time()
event_time_est_16 = event_time_utc_16.astimezone(est_tz).time()
event_time_est_EoD = event_time_utc_EoD.astimezone(est_tz).time()

In [163]:
#Determine outcome based on time of day
utc_now = datetime.now(timezone.utc)  # Global time in UTC
est_now = utc_now.astimezone(est_tz).time()  # Convert to EST and extract

if est_now > event_time_est_0 and est_now < event_time_est_6:
  artD.to_csv('DayFiles/DayFile_' + sourceSuffix + '.tsv', sep="\t") #Define DayFile

elif est_now > event_time_est_6 and est_now < event_time_est_12:
  i = 1+2
  #Load DayFile, find unq, and append with new dataframe
  #Save over previous DayFile

elif est_now > event_time_est_12 and est_now < event_time_est_16:
  j = 1+2
  #Load DayFile, find unq, and append with new dataframe
  #Save over previous DayFile

elif est_now > event_time_est_16 and est_now < event_time_est_EoD:
  w = 1+2
  #Load Master file
  #Load DayFile, find unq, and append with new dataframe
  #Find unq Day and Master
  #Append Master
  #Save new makster

In [154]:
  # my_file = Path('DayFiles/DayFile_' + sourceSuffix + '.tsv')
  # if my_file.is_file():

False